# String Formatting #

It is easy to convert just about any Python object to a string representation, using one of the [`str()`](https://docs.python.org/3/library/functions.html#func-str), [`repr()`](https://docs.python.org/3/library/functions.html#repr) or [`ascii()`](https://docs.python.org/3/library/functions.html#ascii) functions. But sometimes you need more control, particularly over field widths, numbers of digits/characters, sign representations and the like.

This is where *formatted* string output comes in. As of version 3.6, Python offers no less than *three* different ways, built into the language, of doing formatted string output:
* the traditional [C-style](http://man7.org/linux/man-pages/man3/printf.3.html) [`printf`](https://docs.python.org/3/library/stdtypes.html#old-string-formatting) form, predating Python 3.0
* the [`str.format()`](https://docs.python.org/3/library/string.html#formatstrings) method
* [formatted string literals](https://docs.python.org/3/whatsnew/3.6.html#pep-498-formatted-string-literals), reminiscent of those in Perl, introduced in Python 3.6.

The rest of this notebook is basically a series of random observations on various salient points that have struck me about these various formatting mechanisms.

## `printf`-Style Formatting ##

`printf`-style formatting originated with C, but is widely supported in some form across a number of languages, not just Python.

Python doesn’t actually provide a direct `printf` equivalent: instead, it provides the equivalent of `sprintf`, which returns the formatted string as the result, in the form of the “`%`” operator with a string as its left operand, and the sequence of items to be substituted as its right operand. It is then up to you to do what you want with this string: you can print it out, or assign it to something for later use, or manipulate it further in an expression.

Also, Python does not allow the use of the “`$`” character for indicating the indexes of items to be substituted. But it has its own keyword-based alternative (see below).

### Field Width Versus Number Of Digits ###

Supposing you want to output an integer value

In [ ]:
val1 = 3

as 3 digits with leading zeroes as necessary. It is quite common to write

In [ ]:
"%03d" % val1

and while this works in this simple situation, note that the “3” in the format string specifies the *total field width*, not the *number of digits*. This becomes apparent when you add other format embellishments, such as an explicit sign:

In [ ]:
"%+03d" % val1

As you can see, these take up space allocated to the total field width. In order to get the correct number of digits in all situations, you have to specify it *after a decimal point*:

In [ ]:
"%0+.3d" % val1

### Variable Field Widths And Precisions ###

It is possible for the field width and/or precision values to be specified as “`*`”, which means they are taken from items in the format list, immediately preceding the item being formatted:

In [ ]:
nr_digits = 5
"%0+.*d" % (nr_digits, val1)

### Items By Keyword ###

One useful enhancement that Python offers is the ability to specify substitution items by keyword. In this case, the right operand is not a sequence, but a dictionary mapping those keywords to the corresponding items. This is convenient for allowing the items to be specified in a different order, and also for items to be given once but appear at multiple places in the output string.

In [ ]:
"%(val)0+.3d" % {"val" : val1}

### Multilevel Formatting ###

However, the keyword option is not available for variable field widths or precisions. But there is a way around that, by applying more than one level of string formatting:

In [ ]:
val2 = 99
"%%0+.%(nr_digits)dd %%0+.%(nr_digits)dd" \
    % {"nr_digits" : nr_digits} \
    % (val1, val2)

## `str.format()` ##

One obvious advantage of this newer-style string formatting mechanism is that you don’t need to specify the item type (e.g. “`d`”, “`f`” etc) at all: you can still do so, but otherwise reasonable defaults will be used.

This solves some of the limitations of `printf`-style formatting, such as allowing referencing items by index, and being able to specify variable widths and precisions by index or keyword. In addition, it is possible to access attributes of the items, or index them in simple ways.

In [ ]:
from math import \
    pi as π

"{val:.{nr_digits}f}".format(nr_digits = nr_digits, val = π)

This mechanism also allows types to define their own `__format__()` methods, which can implement an entire “mini-language” to be used in the format string to control the representation of values of that type. [For example](https://docs.python.org/3/library/string.html#format-examples), the `datetime.datetime` class [uses this](https://docs.python.org/3/library/datetime.html#datetime.date.__format__) to bring the entire range of options available in in the [`strftime()`](https://docs.python.org/3/library/datetime.html#datetime.date.strftime) function into format strings.

However, one peculiar omission (at least as of Python 3.6) is the inexplicable lack of support of precision specifiers for integer types. Adapting our previous example of the distinction between field widths and number of decimal digits:

In [ ]:
"{:+03d}".format(val1)

is valid, but

In [ ]:
"{:+0.3d}".format(val1)

is not.

## Formatted String Literals ##

[These](https://docs.python.org/3/reference/lexical_analysis.html#f-strings) use a similar format mini-language to `str.format()` formatting, except that they take the form of a string literal with a special prefix, which can contain direct embedded references to variables accessible in the current environment.

In [ ]:
f"val1 = {val1}, val2 = {val2}"